<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://join.slack.com/t/arize-ai/shared_invite/zt-1px8dcmlf-fmThhDFD_V_48oU7ALan4Q">Community</a>
    </p>
</center>
<h1 align="center">Evaluate RAG with LLM Evals</h1>

In this tutorial we will look into building a RAG pipeline and evaluating it with Phoenix Evals.

It has the the following sections:

1. Understanding Retrieval Augmented Generation (RAG).
2. Building RAG (with the help of a framework such as LlamaIndex and LLM providers like Mistral).
3. Evaluating RAG with Evals.

## Retrieval Augmented Generation (RAG)

LLMs are trained on vast datasets, but these will not include your specific data (things like company knowledge bases and documentation). Retrieval-Augmented Generation (RAG) addresses this by dynamically incorporating your data as context during the generation process. This is done not by altering the training data of the LLMs but by allowing the model to access and utilize your data in real-time to provide more tailored and contextually relevant responses.

In RAG, your data is loaded and prepared for queries. This process is called indexing. User queries act on this index, which filters your data down to the most relevant context. This context and your query then are sent to the LLM along with a prompt, and the LLM provides a response.

RAG is a critical component for building applications such a chatbots or agents and you will want to know RAG techniques on how to get data into your application.

<img src="https://storage.googleapis.com/arize-phoenix-assets/assets/images/RAG_Pipeline.png">

## Stages within RAG

There are five key stages within RAG, which will in turn be a part of any larger RAG application.

- **Loading**: This refers to getting your data from where it lives - whether it's text files, PDFs, another website, a database or an API - into your pipeline.
- **Indexing**: This means creating a data structure that allows for querying the data. For LLMs this nearly always means creating vector embeddings, numerical representations of the meaning of your data, as well as numerous other metadata strategies to make it easy to accurately find contextually relevant data.
- **Storing**: Once your data is indexed, you will want to store your index, along with any other metadata, to avoid the need to re-index it.

- **Querying**: For any given indexing strategy there are many ways you can utilize LLMs and data structures to query, including sub-queries, multi-step queries, and hybrid strategies. 
- **Evaluation**: A critical step in any pipeline is checking how effective it is relative to other strategies, or when you make changes. Evaluation provides objective measures on how accurate, faithful, and fast your responses to queries are.


## Build a RAG system 

Now that we have understood the stages of RAG, let's build a pipeline. We will use [LlamaIndex](https://www.llamaindex.ai/) for RAG and [Phoenix Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) for evaluation.


In [2]:
!pip install -qq "arize-phoenix" "arize-phoenix-evals>=0.4.0" "llama-index==0.10.19" "llama-index-llms-mistralai" "llama-index-embeddings-mistralai"  "openinference-instrumentation-llama-index>=1.0.0" "llama-index-callbacks-arize-phoenix>=0.1.2" gcsfs nest_asyncio

In [1]:
# The nest_asyncio module enables the nesting of asynchronous functions within an already running async loop.
# This is necessary because Jupyter notebooks inherently operate in an asynchronous loop.
# By applying nest_asyncio, we can run additional async functions within this existing loop without conflicts.
import nest_asyncio

nest_asyncio.apply()

import os
from getpass import getpass

import pandas as pd
import phoenix as px
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex, set_global_handler
from llama_index.core.node_parser import SimpleNodeParser
from llama_index.embeddings.mistralai import MistralAIEmbedding
from llama_index.llms.mistralai import MistralAI
from phoenix.trace import using_project

During this tutorial, we will capture all the data we need to evaluate our RAG pipeline using Phoenix Tracing. To enable this, simply start the phoenix application and instrument LlamaIndex.

In [2]:
# Setup projects to collect tracing under
os.environ["PHOENIX_PROJECT_NAME"] = "mistral-rag" # Collect traces under the project "mistral-rag"
INDEXING_PROJECT = "indexing" # For llama-index indexing
TESTSET_PROJECT = "testset" # For capturing synthetic testset traces

In [3]:
px.launch_app()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


2024-03-20 13:20:59,568 ERROR strawberry.execution: Unknown project: UHJvamVjdDoy

GraphQL request:4:3
3 | ) {
4 |   node(id: $id) {
  |   ^
5 |     __typename
Traceback (most recent call last):
  File "/Users/mikeldking/work/phoenix/.venv/lib/python3.10/site-packages/graphql/execution/execute.py", line 521, in execute_field
    result = resolve_fn(source, info, **args)
  File "/Users/mikeldking/work/phoenix/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 692, in _resolver
    return _get_result_with_extensions(
  File "/Users/mikeldking/work/phoenix/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 678, in extension_resolver
    return reduce(
  File "/Users/mikeldking/work/phoenix/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", line 673, in wrapped_get_result
    return _get_result(
  File "/Users/mikeldking/work/phoenix/.venv/lib/python3.10/site-packages/strawberry/schema/schema_converter.py", l

In [4]:
set_global_handler("arize_phoenix")

For this tutorial we will be using OpenAI for creating synthetic data as well as for evaluation. 

In [5]:
if not (mistral_api_key := os.getenv("MISTRAL_API_KEY")):
    mistral_api_key = getpass("🔑 Enter your MISTRAL API key: ")
os.environ["MISTRAL_API_KEY"] = mistral_api_key

### Load Data and Build an Index

Let's use an [essay by Paul Graham](https://www.paulgraham.com/worked.html) to build our RAG pipeline.

In [6]:
import tempfile
from urllib.request import urlretrieve

with tempfile.NamedTemporaryFile() as tf:
    urlretrieve(
        "https://raw.githubusercontent.com/Arize-ai/phoenix-assets/main/data/paul_graham/paul_graham_essay.txt",
        tf.name,
    )
    documents = SimpleDirectoryReader(input_files=[tf.name]).load_data()

In [7]:
# Define an LLM
llm = MistralAI(model="mistral-large-latest")
Settings.llm = llm
Settings.embed_model = MistralAIEmbedding()

with using_project("indexing"): # Collect traces under the project "indexing"
    # Build index with a chunk_size of 512
    node_parser = SimpleNodeParser.from_defaults(chunk_size=512)
    nodes = node_parser.get_nodes_from_documents(documents)
    vector_index = VectorStoreIndex(nodes)

Build a QueryEngine and start querying.

In [8]:
query_engine = vector_index.as_query_engine()

In [9]:
response_vector = query_engine.query("What did the author do growing up?")

Check the response that you get from the query.

In [10]:
response_vector.response

"The author didn't have a natural talent for drawing in high school, but he was closer to the tribe of kids who could draw than those seeking a signature style. He later attended the Rhode Island School of Design (RISD) where he learned a lot in a color class, but mostly taught himself to paint. In 1993, he dropped out of RISD and moved to New York to pursue his art, becoming a New York artist in the technical sense of making paintings and living in New York. He was nervous about money and decided to write another book on Lisp to supplement his income, hoping to live frugally off the royalties and spend all his time painting."

By default LlamaIndex retrieves two similar nodes/ chunks. You can modify that in `vector_index.as_query_engine(similarity_top_k=k)`.

Let's check the text in each of these retrieved nodes.

In [11]:
# First retrieved node
response_vector.source_nodes[0].get_text()

"I certainly did. So at the end of the summer Dan and I switched to working on this new dialect of Lisp, which I called Arc, in a house I bought in Cambridge.\n\nThe following spring, lightning struck. I was invited to give a talk at a Lisp conference, so I gave one about how we'd used Lisp at Viaweb. Afterward I put a postscript file of this talk online, on paulgraham.com, which I'd created years before using Viaweb but had never used for anything. In one day it got 30,000 page views. What on earth had happened? The referring urls showed that someone had posted it on Slashdot. [10]\n\nWow, I thought, there's an audience. If I write something and put it on the web, anyone can read it. That may seem obvious now, but it was surprising then. In the print era there was a narrow channel to readers, guarded by fierce monsters known as editors. The only way to get an audience for anything you wrote was to get it published as a book, or in a newspaper or magazine. Now anyone could publish anyt

In [12]:
# Second retrieved node
response_vector.source_nodes[1].get_text()

"I was not one of the kids who could draw in high school, but at RISD I was definitely closer to their tribe than the tribe of signature style seekers.\n\nI learned a lot in the color class I took at RISD, but otherwise I was basically teaching myself to paint, and I could do that for free. So in 1993 I dropped out. I hung around Providence for a bit, and then my college friend Nancy Parmet did me a big favor. A rent-controlled apartment in a building her mother owned in New York was becoming vacant. Did I want it? It wasn't much more than my current place, and New York was supposed to be where the artists were. So yes, I wanted it! [7]\n\nAsterix comics begin by zooming in on a tiny corner of Roman Gaul that turns out not to be controlled by the Romans. You can do something similar on a map of New York City: if you zoom in on the Upper East Side, there's a tiny corner that's not rich, or at least wasn't in 1993. It's called Yorkville, and that was my new home. Now I was a New York art

Remember that we are using Phoenix Tracing to capture all the data we need to evaluate our RAG pipeline. You can view the traces in the phoenix application.

In [13]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


We can access the traces by directly pulling the spans from the phoenix session.

In [14]:
spans_df = px.Client().get_spans_dataframe()

In [15]:
spans_df[["name", "span_kind", "attributes.input.value", "attributes.retrieval.documents"]].head()

,name,span_kind,attributes.input.value,attributes.retrieval.documents
context.span_id,,,,
7fc7e248de69a46b,llm,LLM,NaN,NaN
a7f75bfaccbc9b15,chunking,CHAIN,NaN,NaN
32270b7ac9eee5f2,chunking,CHAIN,NaN,NaN
cd07c7eba44b349f,synthesize,CHAIN,What did the author do growing up?,NaN
f551c60080ae6be3,embedding,EMBEDDING,NaN,NaN


Note that the traces have captured the documents that were retrieved by the query engine. This is nice because it means we can introspect the documents without having to keep track of them ourselves.

In [16]:
spans_with_docs_df = spans_df[spans_df["attributes.retrieval.documents"].notnull()]

In [17]:
spans_with_docs_df[["attributes.input.value", "attributes.retrieval.documents"]].head()

,attributes.input.value,attributes.retrieval.documents
context.span_id,,
2d51fccd2d7a5403,What did the author do growing up?,[{'document.content': 'I certainly did. So at ...


We have built a RAG pipeline and also have instrumented it using Phoenix Tracing. We now need to evaluate it's performance. We can assess our RAG system/query engine using Phoenix's LLM Evals. Let's examine how to leverage these tools to quantify the quality of our retrieval-augmented generation system.

## Evaluation

Evaluation should serve as the primary metric for assessing your RAG application. It determines whether the pipeline will produce accurate responses based on the data sources and range of queries.

While it's beneficial to examine individual queries and responses, this approach is impractical as the volume of edge-cases and failures increases. Instead, it's more effective to establish a suite of metrics and automated evaluations. These tools can provide insights into overall system performance and can identify specific areas that may require scrutiny.

In a RAG system, evaluation focuses on two critical aspects:

- **Retrieval Evaluation**: To assess the accuracy and relevance of the documents that were retrieved
- **Response Evaluation**: Measure the appropriateness of the response generated by the system when the context was provided.

### Generate Question Context Pairs

For the evaluation of a RAG system, it's essential to have queries that can fetch the correct context and subsequently generate an appropriate response.

For this tutorial, let's use Phoenix's `llm_generate` to help us create the question-context pairs.

First, let's create a dataframe of all the document chunks that we have indexed.

In [18]:
# Let's construct a dataframe of just the documents that are in our index
document_chunks_df = pd.DataFrame({"text": [node.get_text() for node in nodes]})
document_chunks_df.head()

,text
0,What I Worked On\n\nFebruary 2021\n\nBefore co...
1,I was puzzled by the 1401. I couldn't figure o...
2,I remember vividly how impressed and envious I...
3,I couldn't have put this into words when I was...
4,The default language at Cornell was a Pascal-l...


Now that we have the document chunks, let's prompt an LLM to generate us 3 questions per chunk. Note that you could manually solicit questions from your team or customers, but this is a quick and easy way to generate a large number of questions.

In [19]:
generate_questions_template = """\
Context information is below.

---------------------
{text}
---------------------

Given the context information and not prior knowledge.
generate only questions based on the below query.

You are a Teacher/ Professor. Your task is to setup \
3 questions for an upcoming \
quiz/examination. The questions should be diverse in nature \
across the document. Restrict the questions to the \
context information provided."

Output the questions in JSON format with the keys question_1, question_2, question_3.
"""

In [22]:
import json

from phoenix.evals import MistralAIModel, llm_generate


def output_parser(response: str, index: int):
    try:
        return json.loads(response)
    except json.JSONDecodeError as e:
        return {"__error__": str(e)}


with using_project(TESTSET_PROJECT): # Collect traces under the project "testset"
    questions_df = llm_generate(
        dataframe=document_chunks_df,
        template=generate_questions_template,
        model=MistralAIModel(
            model="mistral-large-latest"
        ),
        output_parser=output_parser,
        concurrency=20,
    )

llm_generate |          | 0/61 (0.0%) | ⏳ 00:00<? | ?it/s

In [23]:
questions_df.head()

,question_1,question_2,question_3
0,What type of stories did the author write befo...,Describe the author's first experience with pr...,What were the limitations the author faced whi...
1,What was the first microcomputer the author's ...,"What was the author's first personal computer,...",What were some of the programs the author wrot...
2,What type of computer did the speaker convince...,Why did the speaker initially plan to study ph...,What were the two specific influences that mad...
3,What was the name of the novel by Heinlein tha...,What programming language was regarded as the ...,What was the subject of the individual's under...
4,What programming language did the author learn...,What was the subject of the author's undergrad...,What realization did the author come to during...


In [25]:
# Construct a dataframe of the questions and the document chunks
questions_with_document_chunk_df = pd.concat([questions_df, document_chunks_df], axis=1)
questions_with_document_chunk_df = questions_with_document_chunk_df.melt(
    id_vars=["text"], value_name="question"
).drop("variable", axis=1)
# If the above step was interrupted, there might be questions missing. Let's run this to clean up the dataframe.
questions_with_document_chunk_df = questions_with_document_chunk_df[
    questions_with_document_chunk_df["question"].notnull()
]

The LLM has generated three questions per chunk. Let's take a quick look.

In [26]:
questions_with_document_chunk_df.head(10)

,text,question
0,What I Worked On\n\nFebruary 2021\n\nBefore co...,What type of stories did the author write befo...
1,I was puzzled by the 1401. I couldn't figure o...,What was the first microcomputer the author's ...
2,I remember vividly how impressed and envious I...,What type of computer did the speaker convince...
3,I couldn't have put this into words when I was...,What was the name of the novel by Heinlein tha...
4,The default language at Cornell was a Pascal-l...,What programming language did the author learn...
5,"I applied to 3 grad schools: MIT and Yale, whi...","What led the author to realize that AI, as pra..."
6,So I looked around to see what I could salvage...,What programming language did the author decid...
7,"And indeed, it would seem very feeble work. On...",What realization did the author have at the Ca...
8,And as an artist you could be truly independen...,What was the author's initial perception about...
9,I remember when my friend Robert Morris got ki...,Why did the author decide to write his dissert...


### Retrieval Evaluation

We are now prepared to perform our retrieval evaluations. We will execute the queries we generated in the previous step and verify whether or not that the correct context is retrieved.

In [27]:
# loop over the questions and generate the answers
for _, row in questions_with_document_chunk_df.iterrows():
    question = row["question"]
    response_vector = query_engine.query(question)
    print(f"Question: {question}\nAnswer: {response_vector.response}\n")

Question: What type of stories did the author write before college and what were their main characteristics?
Answer: Before college, the author wrote short stories. These stories were not particularly good, lacking a well-defined plot. Instead, they were characterized by strong emotions attributed to the characters, which the author mistakenly believed added depth to the narratives.

Question: What was the first microcomputer the author's friend built from a kit, and who sold this kit?
Answer: The first microcomputer the author's friend built from a kit was sold by Heathkit. The specific model or type of the microcomputer is not mentioned in the provided context.

Question: What type of computer did the speaker convince his father to buy in around 1980, and what were some of the programs he wrote using this computer?
Answer: The speaker convinced his father to buy a TRS-80 computer around 1980. With this computer, he wrote various programs including simple games, a program to predict t

Now that we have executed the queries, we can start validating whether or not the RAG system was able to retrieve the correct context. Let's extract all the retrieved documents from the traces logged to phoenix. (For an in-depth explanation of how to export trace data from the phoenix runtime, consult the [docs](https://docs.arize.com/phoenix/how-to/extract-data-from-spans)).

In [28]:
from phoenix.session.evaluation import get_retrieved_documents

retrieved_documents_df = get_retrieved_documents(px.Client())
retrieved_documents_df

context.trace_id  \
context.span_id  document_position                                     
f487e7f44356f3f4 0                  2492ee478a720d560e59b831d0614780   
                 1                  2492ee478a720d560e59b831d0614780   
e36c757a1ff8c124 0                  43a53dfc3d02fc72dc59c1f919711441   
                 1                  43a53dfc3d02fc72dc59c1f919711441   
5c622e50092e7bb6 0                  8d9736d9cc253e2d6480d7a6e1b15f2c   
...                                                              ...   
e2f0e4ffa898013e 1                  eb1053f80afe0cac635424d0e195d288   
2d15b606b03e4889 0                  ea78f9b47d605e33388816bb1d9d8868   
                 1                  ea78f9b47d605e33388816bb1d9d8868   
2d51fccd2d7a5403 0                  32fc39f2b28eeb4039c5349dbaa1e9ff   
                 1                  32fc39f2b28eeb4039c5349dbaa1e9ff   

                                                                                input  \
context.span_id  document_position                                                      
f487e7f44356f3f4 0                  Based on the author's perspective, what is the...   
                 1                  Based on the author's perspective, what is the...   
e36c757a1ff8c124 0                  Why did the author describe leaving YC as 'lik...   
                 1                  Why did the author describe leaving YC as 'lik...   
5c622e50092e7bb6 0                  Explain the author's perspective on the term '...   
...                                                                               ...   
e2f0e4ffa898013e 1                  What was the first microcomputer the author's ...   
2d15b606b03e4889 0                  What type of stories did the author write befo...   
                 1                  What type of stories did the author write befo...   
2d51fccd2d7a5403 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   

                                                                            reference  \
context.span_id  document_position                                                      
f487e7f44356f3f4 0                  Individually these two phenomena are tedious b...   
                 1                  And at 50 there was some opportunity cost to s...   
e36c757a1ff8c124 0                  Individually these two phenomena are tedious b...   
                 1                  I asked Jessica if she wanted to be president,...   
5c622e50092e7bb6 0                  Customary VC practice had once, like the custo...   
...                                                                               ...   
e2f0e4ffa898013e 1                  To call this a difficult sale would be an unde...   
2d15b606b03e4889 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  At least not the painting department. The text...   
2d51fccd2d7a5403 0                  I certainly did. So at the end of the summer D...   
                 1                  I was not one of the kids who could draw in hi...   

                                    document_score  
context.span_id  document_position                  
f487e7f44356f3f4 0                        0.806157  
                 1                        0.796908  
e36c757a1ff8c124 0                        0.799462  
                 1                        0.784284  
5c622e50092e7bb6 0                        0.823575  
...                                            ...  
e2f0e4ffa898013e 1                        0.747313  
2d15b606b03e4889 0                        0.738245  
                 1                        0.738183  
2d51fccd2d7a5403 0                        0.731376  
                 1                        0.725950  

[368 rows x 4 columns]

Let's now use Phoenix's LLM Evals to evaluate the relevance of the retrieved documents with regards to the query. Note, we've turned on `explanations` which prompts the LLM to explain it's reasoning. This can be useful for debugging and for figuring out potential corrective actions.

In [29]:
from phoenix.evals import (
    RelevanceEvaluator,
    run_evals,
)

relevance_evaluator = RelevanceEvaluator(MistralAIModel(model="mistral-large-latest"))

retrieved_documents_relevance_df = run_evals(
    evaluators=[relevance_evaluator],
    dataframe=retrieved_documents_df,
    provide_explanation=True,
    concurrency=20,
)[0]

run_evals |          | 0/368 (0.0%) | ⏳ 00:00<? | ?it/s

Exception in worker on attempt 1: raised MistralException(message=Unexpected exception (ReadError): )
Requeuing...
Worker timeout, requeuing
Worker timeout, requeuing
Worker timeout, requeuing
Worker timeout, requeuing
Worker timeout, requeuing
Worker timeout, requeuing
Exception in worker on attempt 1: raised MistralException(message=Unexpected exception (ReadError): )
Requeuing...
Worker timeout, requeuing
Exception in worker on attempt 1: raised MistralException(message=Unexpected exception (ReadError): )
Requeuing...
Exception in worker on attempt 1: raised MistralException(message=Unexpected exception (ReadError): )
Requeuing...
Exception in worker on attempt 1: raised MistralException(message=Unexpected exception (ReadError): )
Requeuing...


In [30]:
retrieved_documents_relevance_df.head()

label  score  \
context.span_id  document_position                    
f487e7f44356f3f4 0                  relevant    1.0   
                 1                  relevant    1.0   
e36c757a1ff8c124 0                  relevant    1.0   
                 1                  relevant    1.0   
5c622e50092e7bb6 0                  relevant    1.0   

                                                                          explanation  
context.span_id  document_position                                                     
f487e7f44356f3f4 0                  The reference text discusses the concept of "i...  
                 1                  The reference text discusses the origins of Li...  
e36c757a1ff8c124 0                  The reference text provides information that d...  
                 1                  The reference text does not directly explain w...  
5c622e50092e7bb6 0                  The reference text is relevant to the question...

We can now combine the documents with the relevance evaluations to compute retrieval metrics. These metrics will help us understand how well the RAG system is performing.

In [31]:
documents_with_relevance_df = pd.concat(
    [retrieved_documents_df, retrieved_documents_relevance_df.add_prefix("eval_")], axis=1
)
documents_with_relevance_df

context.trace_id  \
context.span_id  document_position                                     
f487e7f44356f3f4 0                  2492ee478a720d560e59b831d0614780   
                 1                  2492ee478a720d560e59b831d0614780   
e36c757a1ff8c124 0                  43a53dfc3d02fc72dc59c1f919711441   
                 1                  43a53dfc3d02fc72dc59c1f919711441   
5c622e50092e7bb6 0                  8d9736d9cc253e2d6480d7a6e1b15f2c   
...                                                              ...   
e2f0e4ffa898013e 1                  eb1053f80afe0cac635424d0e195d288   
2d15b606b03e4889 0                  ea78f9b47d605e33388816bb1d9d8868   
                 1                  ea78f9b47d605e33388816bb1d9d8868   
2d51fccd2d7a5403 0                  32fc39f2b28eeb4039c5349dbaa1e9ff   
                 1                  32fc39f2b28eeb4039c5349dbaa1e9ff   

                                                                                input  \
context.span_id  document_position                                                      
f487e7f44356f3f4 0                  Based on the author's perspective, what is the...   
                 1                  Based on the author's perspective, what is the...   
e36c757a1ff8c124 0                  Why did the author describe leaving YC as 'lik...   
                 1                  Why did the author describe leaving YC as 'lik...   
5c622e50092e7bb6 0                  Explain the author's perspective on the term '...   
...                                                                               ...   
e2f0e4ffa898013e 1                  What was the first microcomputer the author's ...   
2d15b606b03e4889 0                  What type of stories did the author write befo...   
                 1                  What type of stories did the author write befo...   
2d51fccd2d7a5403 0                                 What did the author do growing up?   
                 1                                 What did the author do growing up?   

                                                                            reference  \
context.span_id  document_position                                                      
f487e7f44356f3f4 0                  Individually these two phenomena are tedious b...   
                 1                  And at 50 there was some opportunity cost to s...   
e36c757a1ff8c124 0                  Individually these two phenomena are tedious b...   
                 1                  I asked Jessica if she wanted to be president,...   
5c622e50092e7bb6 0                  Customary VC practice had once, like the custo...   
...                                                                               ...   
e2f0e4ffa898013e 1                  To call this a difficult sale would be an unde...   
2d15b606b03e4889 0                  What I Worked On\n\nFebruary 2021\n\nBefore co...   
                 1                  At least not the painting department. The text...   
2d51fccd2d7a5403 0                  I certainly did. So at the end of the summer D...   
                 1                  I was not one of the kids who could draw in hi...   

                                    document_score eval_label  eval_score  \
context.span_id  document_position                                          
f487e7f44356f3f4 0                        0.806157   relevant         1.0   
                 1                        0.796908   relevant         1.0   
e36c757a1ff8c124 0                        0.799462   relevant         1.0   
                 1                        0.784284   relevant         1.0   
5c622e50092e7bb6 0                        0.823575   relevant         1.0   
...                                            ...        ...         ...   
e2f0e4ffa898013e 1                        0.747313  unrelated         0.0   
2d15b606b03e4889 0                        0.738245   relevant         1.0   
                 1           

Let's compute Normalized Discounted Cumulative Gain [NCDG](https://en.wikipedia.org/wiki/Discounted_cumulative_gain) at 2 for all our retrieval steps.  In information retrieval, this metric is often used to measure effectiveness of search engine algorithms and related applications.

In [32]:
import numpy as np
from sklearn.metrics import ndcg_score


def _compute_ndcg(df: pd.DataFrame, k: int):
    """Compute NDCG@k in the presence of missing values"""
    n = max(2, len(df))
    eval_scores = np.zeros(n)
    doc_scores = np.zeros(n)
    eval_scores[: len(df)] = df.eval_score
    doc_scores[: len(df)] = df.document_score
    try:
        return ndcg_score([eval_scores], [doc_scores], k=k)
    except ValueError:
        return np.nan


ndcg_at_2 = pd.DataFrame(
    {"score": documents_with_relevance_df.groupby("context.span_id").apply(_compute_ndcg, k=2)}
)

In [33]:
ndcg_at_2

,score
context.span_id,
01a95d0bda5f4093,1.00000
02549453aa793789,1.00000
02973a1c82910962,1.00000
02b32a51678f049a,1.00000
0328faa3b419e8de,1.00000
...,...
f8802fe3b9bc3ee8,1.00000
f896b602cdddd0c8,1.00000
fbe19869319b1576,0.63093


Let's also compute precision at 2 for all our retrieval steps.

In [34]:
precision_at_2 = pd.DataFrame(
    {
        "score": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) / 2
        )
    }
)

In [35]:
precision_at_2

,score
context.span_id,
01a95d0bda5f4093,1.0
02549453aa793789,0.5
02973a1c82910962,1.0
02b32a51678f049a,0.5
0328faa3b419e8de,0.5
...,...
f8802fe3b9bc3ee8,1.0
f896b602cdddd0c8,1.0
fbe19869319b1576,0.5


Lastly, let's compute whether or not a correct document was retrieved at all for each query (e.g. a hit)

In [36]:
hit = pd.DataFrame(
    {
        "hit": documents_with_relevance_df.groupby("context.span_id").apply(
            lambda x: x.eval_score[:2].sum(skipna=False) > 0
        )
    }
)

Let's now view the results in a combined dataframe.

In [37]:
retrievals_df = px.Client().get_spans_dataframe("span_kind == 'RETRIEVER'")
rag_evaluation_dataframe = pd.concat(
    [
        retrievals_df["attributes.input.value"],
        ndcg_at_2.add_prefix("ncdg@2_"),
        precision_at_2.add_prefix("precision@2_"),
        hit,
    ],
    axis=1,
)
rag_evaluation_dataframe

,attributes.input.value,ncdg@2_score,precision@2_score,hit
context.span_id,,,,
f487e7f44356f3f4,"Based on the author's perspective, what is the...",1.00000,1.0,True
e36c757a1ff8c124,Why did the author describe leaving YC as 'lik...,1.00000,1.0,True
5c622e50092e7bb6,Explain the author's perspective on the term '...,1.00000,1.0,True
071839d33583260a,Explain the general lesson learned from the au...,1.00000,0.5,True
b676b47f9b765430,What is the difference between the code editor...,1.00000,1.0,True
...,...,...,...,...
e349f25c783dbe21,What was the name of the novel by Heinlein tha...,1.00000,1.0,True
f8243acefd5fd269,What type of computer did the speaker convince...,1.00000,1.0,True
e2f0e4ffa898013e,What was the first microcomputer the author's ...,1.00000,0.5,True


### Observations

Let's now take our results and aggregate them to get a sense of how well our RAG system is performing.

In [38]:
# Aggregate the scores across the retrievals
results = rag_evaluation_dataframe.mean(numeric_only=True)
results

ncdg@2_score         0.807127
precision@2_score    0.652174
hit                  0.853261
dtype: float64

As we can see from the above numbers, our RAG system is not perfect, there are times when it fails to retrieve the correct context within the first two documents. At other times the correct context is included in the top 2 results but non-relevant information is also included in the context. This is an indication that we need to improve our retrieval strategy. One possible solution could be to increase the number of documents retrieved and then use a more sophisticated ranking strategy (such as a reranker) to select the correct context.

We have now evaluated our RAG system's retrieval performance. Let's send these evaluations to Phoenix for visualization. By sending the evaluations to Phoenix, you will be able to view the evaluations alongside the traces that were captured earlier.

In [40]:
from phoenix.trace import DocumentEvaluations, SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=ndcg_at_2, eval_name="ndcg@2"),
    SpanEvaluations(dataframe=precision_at_2, eval_name="precision@2"),
    DocumentEvaluations(dataframe=retrieved_documents_relevance_df, eval_name="relevance"),
)

In [41]:
ndcg_at_2.head()

,score
context.span_id,
01a95d0bda5f4093,1.0
02549453aa793789,1.0
02973a1c82910962,1.0
02b32a51678f049a,1.0
0328faa3b419e8de,1.0


### Response Evaluation

The retrieval evaluations demonstrates that our RAG system is not perfect. However, it's possible that the LLM is able to generate the correct response even when the context is incorrect. Let's evaluate the responses generated by the LLM.

In [42]:
from phoenix.session.evaluation import get_qa_with_reference

qa_with_reference_df = get_qa_with_reference(px.Client())
qa_with_reference_df

,input,output,reference
context.span_id,,,
f7b0899572931c0b,"Based on the author's perspective, what is the...",The author suggests that the difference betwee...,Individually these two phenomena are tedious b...
faa193410fece9f5,Why did the author describe leaving YC as 'lik...,The author described leaving YC as 'like pulli...,Individually these two phenomena are tedious b...
17f2da759590e9f1,Explain the author's perspective on the term '...,The author expresses a dislike for the term 'd...,"Customary VC practice had once, like the custo..."
f66d8c30407510d9,Explain the general lesson learned from the au...,The general lesson learned from the author's e...,[9] We'd had a code editor in Viaweb for users...
94901e6a75cb5288,What is the difference between the code editor...,The code editor in Viaweb was not an app edito...,[9] We'd had a code editor in Viaweb for users...
...,...,...,...
ddf6cfb6b66b0d8a,What was the name of the novel by Heinlein tha...,The novel that inspired the individual to work...,I remember vividly how impressed and envious I...
5d1439a0d8471cb0,What type of computer did the speaker convince...,The speaker convinced his father to buy a TRS-...,I was puzzled by the 1401. I couldn't figure o...
3d8eeb55abd92063,What was the first microcomputer the author's ...,The first microcomputer the author's friend bu...,I was puzzled by the 1401. I couldn't figure o...


Now that we have a dataset of the question, context, and response (input, reference, and output), we now can measure how well the LLM is responding to the queries. For details on the QA correctness evaluation, see the [LLM Evals documentation](https://docs.arize.com/phoenix/llm-evals/running-pre-tested-evals/q-and-a-on-retrieved-data).

In [43]:
from phoenix.evals import (
    HallucinationEvaluator,
    OpenAIModel,
    QAEvaluator,
    run_evals,
)

qa_evaluator = QAEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))
hallucination_evaluator = HallucinationEvaluator(OpenAIModel(model="gpt-4-turbo-preview"))

qa_correctness_eval_df, hallucination_eval_df = run_evals(
    evaluators=[qa_evaluator, hallucination_evaluator],
    dataframe=qa_with_reference_df,
    provide_explanation=True,
    concurrency=20,
)

run_evals |          | 0/368 (0.0%) | ⏳ 00:00<? | ?it/s

In [44]:
qa_correctness_eval_df.head()

,label,score,explanation
context.span_id,,,
f7b0899572931c0b,correct,1.0,The answer correctly identifies the key differ...
faa193410fece9f5,correct,1.0,The given answer accurately reflects the infor...
17f2da759590e9f1,correct,1.0,The question asks for an explanation of the au...
f66d8c30407510d9,correct,1.0,The given answer accurately captures the gener...
94901e6a75cb5288,correct,1.0,The given answer accurately captures the disti...


In [45]:
hallucination_eval_df.head()

,label,score,explanation
context.span_id,,,
f7b0899572931c0b,factual,0.0,The answer accurately reflects the content and...
faa193410fece9f5,factual,0.0,The answer provided is factual based on the re...
17f2da759590e9f1,factual,0.0,The answer accurately reflects the author's pe...
f66d8c30407510d9,factual,0.0,The answer accurately reflects the information...
94901e6a75cb5288,factual,0.0,The answer provided is factual based on the re...


#### Observations

Let's now take our results and aggregate them to get a sense of how well the LLM is answering the questions given the context.

In [46]:
qa_correctness_eval_df.mean(numeric_only=True)

score    0.929348
dtype: float64

In [47]:
hallucination_eval_df.mean(numeric_only=True)

score    0.048913
dtype: float64

Our QA Correctness score of `0.91` and a Hallucinations score `0.05` signifies that the generated answers are correct ~91% of the time and that the responses contain hallucinations 5% of the time - there is room for improvement. This could be due to the retrieval strategy or the LLM itself. We will need to investigate further to determine the root cause.

Since we have evaluated our RAG system's QA performance and Hallucinations performance, let's send these evaluations to Phoenix for visualization.

In [48]:
from phoenix.trace import SpanEvaluations

px.Client().log_evaluations(
    SpanEvaluations(dataframe=qa_correctness_eval_df, eval_name="Q&A Correctness"),
    SpanEvaluations(dataframe=hallucination_eval_df, eval_name="Hallucination"),
)

We now have sent all our evaluations to Phoenix. Let's go to the Phoenix application and view the results! Since we've sent all the evals to Phoenix, we can analyze the results together to make a determination on whether or not poor retrieval or irrelevant context has an effect on the LLM's ability to generate the correct response.

In [49]:
print("phoenix URL", px.active_session().url)

phoenix URL http://localhost:6006/


## Embeddings Analysis
[Embeddings](https://arize.com/blog-course/embeddings-meaning-examples-and-how-to-compute/) encode the meaning of retrieved documents and user queries. Not only are they an essential part of RAG systems, but they are immensely useful for understanding and debugging LLM application performance.

Phoenix takes the high-dimensional embeddings from your RAG application, reduces their dimensionality, and clusters them into semantically meaningful groups of data. You can then select the metric of your choice (e.g., Ragas-computed faithfulness or answer correctness) to visually inspect the performance of your application and surface problematic clusters. The advantage of this approach is that it provides metrics on granular yet meaningful subsets of your data that help you analyze local, not merely global, performance across a dataset. It's also helpful for gaining intuition around what kind of queries your LLM application is struggling to answer.

In [61]:
# First, let's grab the embeddings from our queries
from phoenix.trace.dsl.helpers import SpanQuery

query_embeddings_df = px.Client().query_spans(
    SpanQuery().explode("embedding.embeddings", query="embedding.text", vector="embedding.vector"),
)
query_embeddings_df.head()

,,query,vector
context.span_id,position,,
9cfe16bba032dc4d,0,"Based on the author's perspective, what is the...","[-0.02996826171875, 0.06658935546875, 0.067138..."
9a9fa0ed21195a43,0,Why did the author describe leaving YC as 'lik...,"[-0.04522705078125, 0.054412841796875, 0.03033..."
30f47917b8ea9404,0,Explain the author's perspective on the term '...,"[-0.039794921875, 0.03741455078125, 0.03497314..."
7367e850dcb81943,0,Explain the general lesson learned from the au...,"[-0.05523681640625, 0.04351806640625, 0.053253..."
e388de5571888143,0,What is the difference between the code editor...,"[-0.0125885009765625, 0.035003662109375, 0.021..."


In [53]:
# Next let's grab the embeddings from our corpus (indexing)
from phoenix.trace.dsl.helpers import SpanQuery

client = px.Client()
corpus_df = px.Client().query_spans(
    SpanQuery().explode(
        "embedding.embeddings",
        text="embedding.text",
        vector="embedding.vector",
    ),
    project_name=INDEXING_PROJECT,
)
corpus_df.head()

text  \
context.span_id  position                                                      
7a1055e618c99ec4 0         file_path: /var/folders/1s/4vdv59n15b1ghg42frd...   
b4ce2d65160323a1 0         file_path: /var/folders/1s/4vdv59n15b1ghg42frd...   
                 1         file_path: /var/folders/1s/4vdv59n15b1ghg42frd...   
                 2         file_path: /var/folders/1s/4vdv59n15b1ghg42frd...   
                 3         file_path: /var/folders/1s/4vdv59n15b1ghg42frd...   

                                                                      vector  
context.span_id  position                                                     
7a1055e618c99ec4 0         [-0.04364013671875, 0.07281494140625, 0.039703...  
b4ce2d65160323a1 0         [-0.055023193359375, 0.079345703125, 0.0482788...  
                 1         [-0.05126953125, 0.071533203125, 0.04840087890...  
                 2         [-0.045745849609375, 0.053070068359375, 0.0385...  
                 3         [-0.04779052734375, 0.054229736328125, 0.03375...

In [62]:
# Let's now merge in our evaluations into the queries
query_embeddings_df = query_embeddings_df.iloc[::-1]
query_df = pd.concat(
    [
        hallucination_eval_df[["score", "label"]].rename(columns={"score": "hallucination_score", "label": "hallucination_label"}).reset_index(drop=True),
        qa_correctness_eval_df[["score", "label"]].rename(columns={"score": "qa_correctness_score", "label": "qa_correctness_label"}).reset_index(drop=True),
        query_embeddings_df[["query", "vector"]].reset_index(drop=True),
    ],
    axis=1,
)
query_df.head()

,hallucination_score,hallucination_label,qa_correctness_score,qa_correctness_label,query,vector
0,0.0,factual,1.0,correct,What did the author do growing up?,"[-0.051177978515625, 0.0357666015625, 0.040527..."
1,0.0,factual,1.0,correct,What type of stories did the author write befo...,"[-0.0513916015625, 0.02825927734375, 0.0482177..."
2,0.0,factual,1.0,correct,What was the first microcomputer the author's ...,"[-0.020782470703125, 0.01148223876953125, 0.03..."
3,0.0,factual,1.0,correct,What type of computer did the speaker convince...,"[-0.048797607421875, 0.045654296875, 0.0026817..."
4,0.0,factual,1.0,correct,What was the name of the novel by Heinlein tha...,"[-0.0313720703125, 0.04644775390625, 0.0225067..."


In [63]:
query_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="query", vector_column_name="vector"
    ),
    # response_column_names="answer",
)
corpus_schema = px.Schema(
    prompt_column_names=px.EmbeddingColumnNames(
        raw_data_column_name="text", vector_column_name="vector"
    )
)
# relaunch phoenix with a primary and corpus dataset to view embeddings
px.close_app()
session = px.launch_app(
    primary=px.Dataset(query_df, query_schema, "query"),
    corpus=px.Dataset(corpus_df.reset_index(drop=True), corpus_schema, "corpus"),
)

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Conclusion

We have explored how to build and evaluate a RAG pipeline using LlamaIndex and Phoenix, with a specific focus on evaluating the retrieval system and generated responses within the pipelines. 

Phoenix offers a variety of other evaluations that can be used to assess the performance of your LLM Application. For more details, see the [LLM Evals](https://docs.arize.com/phoenix/llm-evals/llm-evals) documentation.